In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets,models,transforms
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
class convnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,3,1) ##(((224-2)/1)-1)/2
        self.conv2 = nn.Conv2d(6,16,3,1) ##((((224-2)-1)/2-2)-1)/2
        self.fc1 = nn.Linear(54*54*16,120)
        self.fc2 = nn.Linear(120,32)
        self.fc3 = nn.Linear(32,2)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2,2)
        x = x.view(-1,54*54*16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.fc3(x),dim = 1)
        

In [3]:
device = torch.device('cpu')
model = convnet()
model.load_state_dict(torch.load('V:\\projects\\corona mask detector\\mask.pt',))

<All keys matched successfully>

In [4]:
import cv2
source=cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("V:\\projects\\corona mask detector\\Referance\\haarcascade_frontalface_default.xml")

In [5]:
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,scaleFactor = 1.05,minNeighbors=5)
    for x,y,w,h in faces:
        
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(224,224))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,1,224,224))
        result=model.forward(torch.FloatTensor(reshaped))
        
        label = torch.max(result,axis=1)[1]
        label = label.numpy()
        label = label.item()  

        if label == 0:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(img,'with mask' , (x, y),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
            
        elif label == 1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
            cv2.putText(img,'without mask' , (x, y),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()